In [89]:
import pandas as pd
import numpy as np
import seaborn as sb

data = pd.read_pickle('Loan_Binning.pickle')

data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,APPLICANTINCOME_BIN,COAPPLICANTINCOME_BIN,LOANAMOUNT_BIN,LOAN_AMOUNT_TERM_BIN
2,1,2,0,1,0,3000,0.0,66.0,360.0,1.0,1,1,"(2589.8, 3031.3]","(-0.001, 1040.5]","(16.999, 74.7]","(300.0, 360.0]"
3,1,2,0,0,1,2583,2358.0,120.0,360.0,1.0,1,1,"(2232.2, 2589.8]","(2083.0, 2500.0]","(110.1, 120.0]","(300.0, 360.0]"
4,1,1,0,1,1,6000,0.0,141.0,360.0,1.0,1,1,"(5417.0, 6460.0]","(-0.001, 1040.5]","(138.2, 158.0]","(300.0, 360.0]"
5,1,2,1,1,0,5417,4196.0,267.0,360.0,1.0,1,1,"(4435.8, 5417.0]","(3831.2, 20000.0]","(235.5, 700.0]","(300.0, 360.0]"
6,1,2,0,0,1,2333,1516.0,95.0,360.0,1.0,1,1,"(2232.2, 2589.8]","(1040.5, 1702.0]","(74.7, 96.0]","(300.0, 360.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,1,2,0,1,1,3597,2157.0,119.0,360.0,0.0,2,0,"(3418.0, 3887.5]","(2083.0, 2500.0]","(110.1, 120.0]","(300.0, 360.0]"
495,0,2,1,1,1,3326,913.0,105.0,84.0,1.0,3,1,"(3031.3, 3418.0]","(-0.001, 1040.5]","(96.0, 110.1]","(35.999, 300.0]"
496,1,2,0,0,1,2600,1700.0,107.0,360.0,1.0,2,1,"(2589.8, 3031.3]","(1040.5, 1702.0]","(96.0, 110.1]","(300.0, 360.0]"
498,1,2,1,1,0,2895,0.0,95.0,360.0,1.0,3,1,"(2589.8, 3031.3]","(-0.001, 1040.5]","(74.7, 96.0]","(300.0, 360.0]"


In [90]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['Loan_Status'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w

dt_train = pd.DataFrame()
dt_train['Loan_Status'] = data['Loan_Status']
ATTRS = ['Gender','Married','Dependents','Education','Self_Employed','Credit_History','Property_Area','APPLICANTINCOME_BIN','COAPPLICANTINCOME_BIN','LOANAMOUNT_BIN','LOAN_AMOUNT_TERM_BIN']


NEW_WOE_ATTRS=[]
for attr in ATTRS:
    attr_element = set(data[attr].to_numpy())
    attr_woe = dict()
    for i in attr_element:
        w = WOE(data, attr, i)
        attr_woe[i] = w
        NEW_WOE_ATTRS.append( attr_woe[i])
    print(f'WOE {attr} -> {attr_woe}')
    

    lst = []
    for s in data[attr]:
        woe = attr_woe[s]
        lst.append(woe)

    new_field = attr + '_WOE'
    dt_train[new_field] = lst


dt_train

WOE Gender -> {0: -0.17280179645335886, 1: 0.04434373621229723}
WOE Married -> {1: -0.24512245803298496, 2: 0.16034265007517928}
WOE Dependents -> {0: 0.0196427679720442, 1: -0.029278209200386745}
WOE Education -> {0: -0.30047255311614995, 1: 0.07907968599641366}
WOE Self_Employed -> {0: -0.18796404419303647, 1: 0.03150977823221087}
WOE Credit_History -> {0.0: -3.2816767261072313, 1.0: 0.6041439069268907}
WOE Property_Area -> {1: -0.10307480037697031, 2: -0.437494350680441, 3: 0.46782734982314}
WOE APPLICANTINCOME_BIN -> {Interval(6460.0, 9883.1, closed='right'): 0.4227069145426705, Interval(9883.1, 81000.0, closed='right'): -0.3992731378602432, Interval(3418.0, 3887.5, closed='right'): -0.6953234599825407, Interval(2589.8, 3031.3, closed='right'): 0.2657031657330057, Interval(2232.2, 2589.8, closed='right'): 0.4227069145426705, Interval(5417.0, 6460.0, closed='right'): 0.1823215567939548, Interval(3887.5, 4435.8, closed='right'): 0.11952065555492423, Interval(3031.3, 3418.0, closed='r

,Loan_Status,Gender_WOE,Married_WOE,Dependents_WOE,Education_WOE,Self_Employed_WOE,Credit_History_WOE,Property_Area_WOE,APPLICANTINCOME_BIN_WOE,COAPPLICANTINCOME_BIN_WOE,LOANAMOUNT_BIN_WOE,LOAN_AMOUNT_TERM_BIN_WOE
2,1,0.044344,0.160343,0.019643,0.079080,-0.187964,0.604144,-0.103075,0.265703,-0.080500,-0.149812,0.052510
3,1,0.044344,0.160343,0.019643,-0.300473,0.031510,0.604144,-0.103075,0.422707,0.930451,0.011307,0.052510
4,1,0.044344,-0.245122,0.019643,0.079080,0.031510,0.604144,-0.103075,0.182322,-0.080500,0.265703,0.052510
5,1,0.044344,0.160343,-0.029278,0.079080,-0.187964,0.604144,-0.103075,-0.105361,-0.519559,-0.638165,0.052510
6,1,0.044344,0.160343,0.019643,-0.300473,0.031510,0.604144,-0.103075,0.422707,0.265703,0.160343,0.052510
...,...,...,...,...,...,...,...,...,...,...,...,...
494,0,0.044344,0.160343,0.019643,0.079080,0.031510,-3.281677,-0.437494,-0.695323,0.930451,0.011307,0.052510
495,1,-0.172802,0.160343,-0.029278,0.079080,0.031510,0.604144,0.467827,-0.018349,-0.080500,-0.196332,-0.021979
496,1,0.044344,0.160343,0.019643,-0.300473,0.031510,0.604144,-0.437494,0.265703,0.265703,-0.196332,0.052510
498,1,0.044344,0.160343,-0.029278,0.079080,-0.187964,0.604144,0.467827,0.265703,-0.080500,0.160343,0.052510


In [91]:
Y =  dt_train['Loan_Status'].to_numpy()
x_1 = dt_train['Gender_WOE'].to_numpy()
x_2 = dt_train['Married_WOE'].to_numpy()
x_3 = dt_train['Dependents_WOE'].to_numpy()
x_4 = dt_train['Education_WOE'].to_numpy()
x_5 = dt_train['Self_Employed_WOE'].to_numpy()
x_6 = dt_train['Credit_History_WOE'].to_numpy()
x_7 = dt_train['Property_Area_WOE'].to_numpy()
x_8 = dt_train['APPLICANTINCOME_BIN_WOE'].to_numpy()
x_9 = dt_train['COAPPLICANTINCOME_BIN_WOE'].to_numpy()
x_10 = dt_train['LOANAMOUNT_BIN_WOE'].to_numpy()
x_11 = dt_train['LOAN_AMOUNT_TERM_BIN_WOE'].to_numpy()


array_ones = np.ones(x_1.shape)

X = np.array([array_ones, x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11])
X = np.transpose(X)
X

array([[ 1.        ,  0.04434374,  0.16034265, ..., -0.08050048,
        -0.14981228,  0.05250995],
       [ 1.        ,  0.04434374,  0.16034265, ...,  0.93045087,
         0.01130707,  0.05250995],
       [ 1.        ,  0.04434374, -0.24512246, ..., -0.08050048,
         0.26570317,  0.05250995],
       ...,
       [ 1.        ,  0.04434374,  0.16034265, ...,  0.26570317,
        -0.19633229,  0.05250995],
       [ 1.        ,  0.04434374,  0.16034265, ..., -0.08050048,
         0.16034265,  0.05250995],
       [ 1.        ,  0.04434374, -0.24512246, ..., -0.5195593 ,
         0.26570317,  0.05250995]])

In [92]:
np.matmul(np.transpose(X), X)

array([[ 3.38000000e+02,  4.39432151e-01,  2.29628189e+00,
         3.49236563e-02,  1.29893385e+00,  3.35035389e-01,
        -5.63367364e+00,  8.74994402e+00,  6.21964959e+00,
         8.24405374e+00,  5.87710487e+00,  1.47782711e+00],
       [ 4.39432151e-01,  2.53353632e+00,  2.17128732e+00,
        -1.24884514e-01, -2.14761055e-02, -1.09966866e-01,
        -6.01471308e-01, -7.93536232e-01,  9.91227525e-01,
        -1.04268042e-01, -9.91285421e-01,  2.23676274e-03],
       [ 2.29628189e+00,  2.17128732e+00,  1.30899332e+01,
        -5.77464265e-01, -5.10230171e-01,  2.01792489e-02,
         8.28753042e-01,  1.02554835e+00,  1.51991269e+00,
         2.35227903e+00, -5.07070935e-01,  2.65432885e+00],
       [ 3.49236563e-02, -1.24884514e-01, -5.77464265e-01,
         1.94049009e-01,  7.88562210e-02,  5.61649608e-02,
         2.71629445e-01,  8.34261842e-04,  3.18884763e-01,
         2.24224291e-01,  2.74881237e-01, -1.56846880e-01],
       [ 1.29893385e+00, -2.14761055e-02, -5.1023017

In [93]:
np.linalg.inv(np.matmul(np.transpose(X), X))

array([[ 3.00841430e-03, -4.56917607e-04, -2.54531028e-04,
         4.91932788e-04, -7.34234064e-04, -1.61631374e-04,
         5.64577717e-05, -5.65390091e-04, -4.82968408e-04,
        -4.82091221e-04, -4.34137385e-04, -1.84821531e-04],
       [-4.56917607e-04,  4.77759022e-01, -7.78088014e-02,
         7.99345039e-02, -6.42666639e-03,  2.01419188e-02,
         1.03967493e-04,  9.48048320e-03, -1.02224757e-02,
         2.33258404e-03,  1.13776426e-02,  1.15636417e-02],
       [-2.54531028e-04, -7.78088014e-02,  1.05667287e-01,
         2.71734696e-01,  2.58888126e-03, -9.69322707e-03,
        -3.94381696e-05, -3.86276259e-03, -3.94554810e-03,
        -6.18706889e-03, -9.45702139e-04, -1.17352443e-02],
       [ 4.91932788e-04,  7.99345039e-02,  2.71734696e-01,
         6.30296741e+00, -6.80319228e-02, -1.61436279e-01,
        -1.25665081e-03, -3.31849143e-03, -6.57375053e-02,
        -3.69486301e-02, -3.33488318e-02,  1.92018457e-02],
       [-7.34234064e-04, -6.42666639e-03,  2.5888812

In [94]:
np.matmul(np.transpose(X), Y)

array([230.0, 0.8618014242051484, 4.441600868638101, 0.06602771087895018,
       2.6266859755600276, 0.6630343206510837, 123.40981606104879,
       16.131739967017236, 12.611916423348063, 13.942741609089923,
       11.32099836685913, 5.710250489890591], dtype=object)

In [95]:
B = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.transpose(X), Y))
B

array([0.6674713449605472, 0.22700102735079428, 0.08112605786874723,
       -0.3013484425827287, 0.10661842736806694, 0.17548210787580668,
       0.1760570804162501, 0.1802172918098554, 0.19754959947527875,
       0.137886638212547, 0.13280053587254495, 0.1291943061980887],
      dtype=object)

In [96]:
def predict(x):
    y = B[0] + B[1]*x+ B[2]*x+ B[3]*x+ B[4]*x+ B[5]*x \
    + B[6]*x+ B[7]*x+ B[8]*x+ B[9]*x+ B[10]*x+ B[11]*x
    return y

eq = f'y = {B[0]:.3f}'
for i in range(1, len(B)):
    eq = eq + f'+ ({B[i]:.3f})x_{i}'
print(eq)

y = 0.667+ (0.227)x_1+ (0.081)x_2+ (-0.301)x_3+ (0.107)x_4+ (0.175)x_5+ (0.176)x_6+ (0.180)x_7+ (0.198)x_8+ (0.138)x_9+ (0.133)x_10+ (0.129)x_11


In [97]:
data_test = pd.read_pickle('Data_test.pickle')

attr = pd.read_pickle('Loan_attr.pickle')
attr


,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,APPLICANTINCOME_BIN,"(149.999, 2232.2]",23,11,0.100000,0.101852,-0.018349
1,APPLICANTINCOME_BIN,"(2232.2, 2589.8]",26,8,0.113043,0.074074,0.422707
2,APPLICANTINCOME_BIN,"(2589.8, 3031.3]",25,9,0.108696,0.083333,0.265703
3,APPLICANTINCOME_BIN,"(3031.3, 3418.0]",23,11,0.100000,0.101852,-0.018349
4,APPLICANTINCOME_BIN,"(3418.0, 3887.5]",17,16,0.073913,0.148148,-0.695323
5,APPLICANTINCOME_BIN,"(3887.5, 4435.8]",24,10,0.104348,0.092593,0.119521
6,APPLICANTINCOME_BIN,"(4435.8, 5417.0]",23,12,0.100000,0.111111,-0.105361
7,APPLICANTINCOME_BIN,"(5417.0, 6460.0]",23,9,0.100000,0.083333,0.182322
8,APPLICANTINCOME_BIN,"(6460.0, 9883.1]",26,8,0.113043,0.074074,0.422707
9,APPLICANTINCOME_BIN,"(9883.1, 81000.0]",20,14,0.086957,0.129630,-0.399273


In [98]:
# y_predict
data_test['y_predict'] = B[0] + B[1]*dt_train['Gender_WOE'] + B[2]*dt_train['Married_WOE'] \
        + B[3]*dt_train['Dependents_WOE'] + B[4]*dt_train['Education_WOE'] + B[5]*dt_train['Self_Employed_WOE'] \
        + B[6]*dt_train['Credit_History_WOE'] + B[7]*dt_train['Property_Area_WOE'] + B[8]*dt_train['APPLICANTINCOME_BIN_WOE'] \
        + B[9]*dt_train['COAPPLICANTINCOME_BIN_WOE'] + B[10]*dt_train['LOANAMOUNT_BIN_WOE'] \
        + B[11]*dt_train['LOAN_AMOUNT_TERM_BIN_WOE']

data_test['P'] = 1/(1+np.exp(-data_test['y_predict']))

data_test['predict'] = data_test['P'].map(lambda x: 1 if x >= 0.5 else 0)
data_test

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,y_predict,P,predict
5,1,2,1,1,0,5417,4196.0,267.0,360.0,1.0,1,1,0.592184,0.643866,1
9,1,2,1,1,1,12841,10968.0,349.0,360.0,1.0,3,0,0.675522,0.662739,1
13,1,1,0,1,1,1853,2840.0,114.0,360.0,1.0,2,0,0.670802,0.661683,1
15,1,1,0,1,1,4950,0.0,125.0,360.0,1.0,1,1,0.712420,0.670936,1
16,1,1,1,0,1,3596,0.0,100.0,240.0,1.0,1,1,0.550383,0.634224,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1,2,1,1,1,2873,1872.0,132.0,360.0,0.0,3,0,NaN,NaN,0
480,0,1,0,0,1,4350,0.0,154.0,360.0,1.0,2,1,0.658034,0.658819,1
482,1,2,0,1,1,2083,3150.0,128.0,360.0,1.0,3,1,NaN,NaN,0
488,1,2,1,1,0,4583,2083.0,160.0,360.0,1.0,3,1,0.870637,0.704878,1


In [99]:
n_correct = (data_test['Loan_Status'] == data_test['predict']).sum()
print(n_correct)

accuracy = n_correct/144
print('accuracy is')
print(accuracy)

91
accuracy is
0.6319444444444444
